<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/Maestro_HMA_Buy_Signal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Gerekli kütüphaneleri yükleyin
!pip install git+https://github.com/rongardF/tvdatafeed openpyxl  matplotlib openpyxl TA
!pip install tradingview-screener==2.5.0

import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import ta
import matplotlib.pyplot as plt

# TvDatafeed nesnesi oluşturma
tv = TvDatafeed()

# Türkiye piyasasındaki tüm sembolleri almak için
try:
    Hisseler = get_all_symbols(market='turkey')
    Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
    Hisseler = sorted(Hisseler)
except Exception as e:
    print(f"Hata alındı: {e}")

# HMA hesaplama fonksiyonları
def calculate_hma(data, length):
    wma1 = ta.trend.WMAIndicator(data['hl2'], window=length // 2).wma()
    wma2 = ta.trend.WMAIndicator(data['hl2'], window=length).wma()
    hma1 = ta.trend.WMAIndicator(2 * wma1 - wma2, window=int(round(np.sqrt(length)))).wma()
    p = length // 2
    hma2 = ta.trend.WMAIndicator(ta.trend.WMAIndicator(data['hl2'], window=p // 3).wma() * 3 -
                                  ta.trend.WMAIndicator(data['hl2'], window=p // 2).wma() -
                                  ta.trend.WMAIndicator(data['hl2'], window=p).wma(),
                                  window=p).wma()
    return hma1, hma2

# Crossover sinyal hesaplama fonksiyonu
def calculate_crossover(data, hma1, hma2):
    data['crossDown'] = (hma1 > hma2) & (hma1.shift(1) < hma2.shift(1))
    data['crossUp'] = (hma2 > hma1) & (hma2.shift(1) < hma1.shift(1))
    return data

# Sembole göre veri çekme ve işleme fonksiyonu
def scan_stocks_for_hma(symbols, interval, length=24, n_bars=500):
    results = []

    for symbol in symbols:
        try:
            # TradingView'dan veri çekme
            data = tv.get_hist(symbol=symbol, exchange='BIST', interval=interval, n_bars=n_bars)
            if data is None or data.empty:
                print(f"{symbol} için veri bulunamadı veya boş.")
                continue

            # HMA hesaplama
            data['hl2'] = (data['high'] + data['low']) / 2
            hma1, hma2 = calculate_hma(data, length)
            data['hma1'] = hma1
            data['hma2'] = hma2
            data = calculate_crossover(data, hma1, hma2)

            # Koşulları kontrol etme (crossover)
            if data['crossUp'].iloc[-1]:
                results.append({
                    'Hisse Adı': symbol,
                    'Son Fiyat': data.iloc[-1]['close'],
                    'Zaman Dilimi': interval_name(interval),
                })
                print(f"{symbol} için crossover koşulları sağlanıyor ({interval_name(interval)})")
        except Exception as e:
            print(f"Hata alındı {symbol}: {e}")

    return results

# Zaman dilimi ismini döndürme fonksiyonu
def interval_name(interval):
    interval_map = {
        Interval.in_15_minute: '15 Dakika',
        Interval.in_30_minute: '30 Dakika',
        Interval.in_45_minute: '45 Dakika',
        Interval.in_1_hour: '1 Saat',
        Interval.in_2_hour: '2 Saat',
        Interval.in_4_hour: '4 Saat',
        Interval.in_daily: 'Günlük',
        Interval.in_weekly: 'Haftalık',
        Interval.in_monthly: 'Aylık'
    }
    return interval_map.get(interval, 'Günlük')

# Kullanıcının zaman dilimini seçmesini sağlayan fonksiyon
def get_interval_choice():
    intervals = {
        '1': ('15 Dakika', Interval.in_15_minute),
        '2': ('30 Dakika', Interval.in_30_minute),
        '3': ('45 Dakika', Interval.in_45_minute),
        '4': ('1 Saat', Interval.in_1_hour),
        '5': ('2 Saat', Interval.in_2_hour),
        '6': ('4 Saat', Interval.in_4_hour),
        '7': ('Günlük', Interval.in_daily),
        '8': ('Haftalık', Interval.in_weekly),
        '9': ('Aylık', Interval.in_monthly)
    }
    print("Zaman dilimi seçenekleri:")
    for key, (name, _) in intervals.items():
        print(f"{key}: {name}")

    choice1 = input("Birinci zaman dilimini seçin (1-9): ")
    choice2 = input("İkinci zaman dilimini seçin (1-9): ")

    interval1 = intervals.get(choice1, ('Günlük', Interval.in_daily))[1]
    interval2 = intervals.get(choice2, ('Günlük', Interval.in_daily))[1]

    return interval1, interval2

# Kullanıcının seçimlerini al ve tarama işlemlerini başlat
interval1, interval2 = get_interval_choice()
results_interval1 = scan_stocks_for_hma(Hisseler, interval1, length=24, n_bars=500)
results_interval2 = scan_stocks_for_hma(Hisseler, interval2, length=24, n_bars=500)

# Sonuçları tablo olarak gösterme ve Excel'e yazma
def show_results(results, timeframe, writer):
    if results:
        df_results = pd.DataFrame(results)
        print(f"\n{timeframe} periyodunda koşulları sağlayan hisseler:")
        print(df_results)
        df_results.to_excel(writer, sheet_name=timeframe, index=False)

# Excel dosyasını oluşturma
filename = 'tarama_sonuclari_hma.xlsx'
with pd.ExcelWriter(filename, engine='openpyxl') as writer:
    show_results(results_interval1, interval_name(interval1), writer)
    show_results(results_interval2, interval_name(interval2), writer)

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-_cjaexjk
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-_cjaexjk
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17533 sha256=d7a25e76276c5d6f013bed10e8121274484ef1eb789963ad94bc381ffcb7b053
  Stored in directory: /tmp/pip-ephem-wheel-cache-5ft44b7u/wheels/e4/32/1e/21ebcacc6549d75fae3bf3ff75cee8fcbe4e5c189d88fbd5a1
  Created wheel for TA: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=b243786fe03d726a43369de9b6f1345324a72cb50c6420cd23e9cbd10b439bfa
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built tvdatafeed TA
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5

Zaman dilimi seçenekleri:
1: 15 Dakika
2: 30 Dakika
3: 45 Dakika
4: 1 Saat
5: 2 Saat
6: 4 Saat
7: Günlük
8: Haftalık
9: Aylık
ISGYO için crossover koşulları sağlanıyor (Günlük)


ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


KTSKR için veri bulunamadı veya boş.
SNPAM için crossover koşulları sağlanıyor (Günlük)
TURGG için crossover koşulları sağlanıyor (Günlük)
AHGAZ için crossover koşulları sağlanıyor (1 Saat)
AKENR için crossover koşulları sağlanıyor (1 Saat)
ATAGY için crossover koşulları sağlanıyor (1 Saat)
BEGYO için crossover koşulları sağlanıyor (1 Saat)
BERA için crossover koşulları sağlanıyor (1 Saat)
BRKSN için crossover koşulları sağlanıyor (1 Saat)
DEVA için crossover koşulları sağlanıyor (1 Saat)
DOBUR için crossover koşulları sağlanıyor (1 Saat)
EDATA için crossover koşulları sağlanıyor (1 Saat)
EDIP için crossover koşulları sağlanıyor (1 Saat)
EGGUB için crossover koşulları sağlanıyor (1 Saat)
GLDTR için crossover koşulları sağlanıyor (1 Saat)
GMSTR için crossover koşulları sağlanıyor (1 Saat)
ISBIR için crossover koşulları sağlanıyor (1 Saat)
MEDTR için crossover koşulları sağlanıyor (1 Saat)
MEPET için crossover koşulları sağlanıyor (1 Saat)
MNDTR için crossover koşulları sağlanıyor (1 Saa